# HVM Flow Curve

**Hybrid Vitrimer Model — Subnetwork decomposition and $\sigma_E \to 0$**

## Physical Motivation

The steady-state flow curve reveals the most distinctive vitrimer property:
**the E-network stress vanishes** ($\sigma_E \to 0$) because BER continuously
redefines the natural reference state to match the current deformation. At
steady state, only the permanent network ($\sigma_P = G_P \gamma$) and the
dissociative network ($\sigma_D = \eta_D \dot\gamma$) contribute.

This fundamentally distinguishes vitrimers from:
- **Permanent elastomers**: All elastic stress persists (no relaxation)
- **Thermoplastics**: All stress relaxes to zero (no permanent network)
- **Vitrimers**: Exchangeable stress vanishes, but permanent stress persists

Experimentally, the flow curve is measured as stress vs. shear rate at long
times. For vitrimers, the D-network viscosity $\eta_D = G_D/k_d^D$ determines
the rate-dependent dissipation, while $G_P$ sets the elastic baseline.

> **Handbook:** See [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) for the steady-state solution and subnetwork architecture. See [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) for practical flow curve interpretation.

## Learning Objectives

- Understand steady-state flow: $\sigma_E = 0$ as natural state tracks deformation
- Decompose total stress into subnetwork contributions
- Observe Newtonian behavior of D-network: $\sigma_D = \eta_D \dot\gamma$
- Analyze apparent viscosity and its plateau at $\eta_D$

## Prerequisites

- **Notebook 01** (SAOS) — HVM architecture and parameter meanings

## Estimated Runtime

- ~30 seconds

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()

from rheojax.models.hvm import HVMLocal
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## Theory: Steady-State Flow

At steady state under constant shear rate γ̇, the HVM model exhibits:

$$\sigma_{ss} = \sigma_P + \sigma_E + \sigma_D$$

where:

1. **P-network (permanent)**: σ_P = G_P·γ (unbounded growth with strain)
2. **E-network (exchangeable)**: σ_E → 0 (natural state tracks deformation via TST)
3. **D-network (dangling)**: σ_D = η_D·γ̇ with η_D = G_D/k_d_D (Newtonian viscosity)

The E-network stress vanishes at steady state because TST-mediated bond exchange allows the natural reference state to continuously evolve with the deformation. This is the key feature distinguishing vitrimers from permanent elastomers.

The apparent viscosity is:

$$\eta_{app}(\dot{\gamma}) = \frac{\sigma_{ss}}{\dot{\gamma}} = G_P \frac{\gamma}{\dot{\gamma}} + \eta_D$$

For transient startup, γ/γ̇ → t, so η_app grows linearly at short times before plateauing at η_D.

In [ ]:
# Model setup
model = HVMLocal()
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-5)    # Activation volume [m³/mol]

eta_D = model.G_D / model.k_d_D

print("HVM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  k_d_D = {model.k_d_D:.1f} 1/s")
print(f"  η_D = G_D/k_d_D = {eta_D:.1f} Pa·s")
print(f"  T = {model.T:.0f} K")


In [ ]:
# Flow curve with component decomposition
gamma_dot = np.logspace(-3, 2, 100)

print("Computing flow curve with subnetwork decomposition...")
result = model.predict_flow_curve(gamma_dot, return_components=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Flow curve: stress vs shear rate
ax1.loglog(gamma_dot, result["stress"], "k-", lw=2, label="Total σ")
ax1.loglog(gamma_dot, result["sigma_P"], "b--", lw=1.5, label="σ_P (permanent)")
ax1.loglog(gamma_dot, result["sigma_D"], "r--", lw=1.5, label="σ_D = η_D·γ̇")
ax1.loglog(gamma_dot, np.abs(result["sigma_E"]) + 1e-10, "g:", lw=1.5, 
           label="|σ_E| (≈0 at steady state)", alpha=0.7)
ax1.set_xlabel("γ̇ [1/s]")
ax1.set_ylabel("Stress σ [Pa]")
ax1.set_title("Flow Curve: Subnetwork Decomposition")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Apparent viscosity
eta_app = result["stress"] / gamma_dot
ax2.loglog(gamma_dot, eta_app, "k-", lw=2, label="η_app")
ax2.axhline(eta_D, color="r", ls=":", lw=1.5, 
            label=f"η_D = G_D/k_d_D = {eta_D:.1f} Pa·s")
ax2.set_xlabel("γ̇ [1/s]")
ax2.set_ylabel("η [Pa·s]")
ax2.set_title("Apparent Viscosity")
ax2.legend()
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nStress range: {result['stress'].min():.2f} to {result['stress'].max():.2f} Pa")
print(f"σ_E/σ_total at γ̇ = 1 s⁻¹: {result['sigma_E'][50]/result['stress'][50]:.2e}")
print(f"(σ_E ≈ 0 confirms steady-state natural state evolution)")

## Key Takeaways

1. **E-network vanishes**: $\sigma_E \to 0$ at steady state — the vitrimer signature
2. **P-network dominance**: $\sigma_P = G_P \gamma$ grows unbounded (permanent crosslinks store elastic energy)
3. **Newtonian D-network**: $\sigma_D = \eta_D \dot\gamma$ provides rate-dependent dissipation
4. **Apparent viscosity**: Dominated by $\eta_D$ at high shear rates
5. **Practical implication**: Real vitrimers undergo network damage at large strains to limit $\sigma_P$

## Further Reading

- [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) — Steady-state derivation and protocol summary
- [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) — Vitrimer vs conventional transient network comparison table

**References:**
1. Vernerey, F.J., Long, R. & Brighenti, R. (2017). *J. Mech. Phys. Solids*, 107, 1-20.
2. Montarnal, D. et al. (2011). "Silica-like malleable materials from permanent organic networks." *Science*, 334, 965-968.

## Next Notebooks

- **Notebook 06**: LAOS — nonlinear harmonic generation from TST
- **Notebooks 08-13**: Full NLSQ + NUTS Bayesian inference pipeline